Import all required libraries

In [1]:
from die import Die
from grid import grid
from numpy import array, ndarray

Define the backtracking function

In [2]:
def backtrack(row: int,
              col: int,
              die: Die,
              score: int,
              turn: int,
              matrix: array,
              visited_cells: dict[tuple[int, int], set[str]]) -> bool:
    """
    Returns true if a satisfiable path has been found.
    """
    # Check if you've already visited this cell with the same orientation
    orientation: str = die.serialize()
    if (row, col) not in visited_cells:
        visited_cells[(row, col)]: set[str] = set()

    if orientation in visited_cells[(row, col)]:
        return False  # You've made no progress

    # Remember the value of this cell
    c: int = matrix[row][col]

    # Check: Is there a number on this die-face?
    index_of_top_face: int = die.index_of_top_face()

    # Remember whether the top-face was set during this function call
    top_face_number_set_here: bool = False

    if turn > 0:
        # Only check the equation after you've started moving
        if not die.die_face_has_number(index_of_top_face):
            # Since c = s + t x n, n exists iff t divides c - s
            if (c - score) % turn == 0:
                die.set_die_face_number(index_of_top_face, (c - score) // turn)
                top_face_number_set_here = True
            else:
                # No integer will work as the die-face
                return False

        # Check: Given the number on the top-face, does the equation still hold?
        if c != score + turn * die.get_die_face_number(index_of_top_face):
            # We only enter this conditional if the die-face number was not set
            # in this function call, because if it was, n would have been chosen
            # by construction to satisfy the equation.
            #
            # Thus, we don't delete the die-face number here, because this face might be
            # valid, just that the right path does not include this cell.
            return False

    visited_cells[(row, col)].add(orientation)

    # Check: Are we on the last square?
    if row == 0 and col == 5:
        matrix[row][col] = 0 # Delete this number from the sum
        return True

    # Check: Does rolling left work?
    die.tip_left()
    if col > 0 and backtrack(row, col - 1, die, c, turn + 1, matrix, visited_cells):
        matrix[row][col - 1] = 0
        return True
    die.tip_right()

    # Check: Does rolling right work?
    die.tip_right()
    if col + 1 < len(matrix[0]) and backtrack(row, col + 1, die, c, turn + 1, matrix, visited_cells):
        matrix[row][col + 1] = 0
        return True
    die.tip_left()

    # Check: Does rolling up work?
    die.tip_up()
    if row > 0 and backtrack(row - 1, col, die, c, turn + 1, matrix, visited_cells):
        matrix[row - 1][col] = 0
        return True
    die.tip_down()

    # Check: Does rolling down work?
    die.tip_down()
    if row + 1 < len(matrix) and backtrack(row + 1, col, die, c, turn + 1, matrix, visited_cells):
        matrix[row + 1][col] = 0
        return True
    die.tip_up()

    # Everything failed. If you had set a die-face here, reset it.
    if top_face_number_set_here:
        die.die_face_numbers[index_of_top_face] = 0

    # Deregister yourself from visited_cells
    visited_cells[(row, col)].remove(orientation)
    return False


Print the matrix before starting for comparison's sake

In [3]:
print(grid)

[[ 57  33 132 268 492 732]
 [ 81 123 240 443 353 508]
 [186  42 195 704 452 228]
 [ -7   2 357 452 317 395]
 [  5  23  -4 592 445 620]
 [  0  77  32 403 337 452]]


Run the backtracker

In [4]:
visited_cells: dict[tuple[int, int], set[str]] = {}
print("Success" if backtrack(row=5, col=0, die=Die(), score=0, turn=0, matrix=grid, visited_cells=visited_cells) else "Failure")

Failure


Set the cells to 0 if the die has visited it before

In [5]:
print(grid)
print(visited_cells)

[[ 57  33 132 268 492 732]
 [ 81 123 240 443 353 508]
 [186  42 195 704 452 228]
 [ -7   2 357 452 317 395]
 [  5  23  -4 592 445 620]
 [  0  77  32 403 337 452]]
{(5, 0): set(), (5, 1): set(), (5, 2): set(), (4, 1): set(), (4, 0): set(), (3, 0): set(), (3, 1): set(), (2, 0): set(), (4, 2): set(), (4, 3): set(), (4, 4): set(), (3, 3): set(), (3, 2): set(), (3, 4): set(), (2, 3): set(), (5, 3): set(), (2, 1): set(), (2, 2): set(), (1, 1): set(), (1, 0): set(), (1, 2): set(), (0, 1): set(), (0, 0): set(), (0, 2): set(), (0, 3): set(), (5, 4): set(), (5, 5): set(), (4, 5): set(), (3, 5): set(), (2, 4): set(), (2, 5): set(), (1, 3): set()}
